# Lab-93 Socket-Based Communication System
This is a custom improvement on the basic socket-server chat system
everyone does in python at some point to get their feet wet; meant to
be integrated as an actual sub-system within the Lab-93 ecosystem.

## Upgraded Implementation
Some of the things being improved upon is synchronicity; the Lab-93 version
contains a database for storing messages while users are away from the client.

Another thing being introduced is in-line commands; by starting a message with
a forward slash character simple pre-defined commands can be executed:
  - ```/quit```: Exits the client.
  - ```/subject```: Attaches a subject to the message.
  - ```/scan```: Searches the local area network for other instances.

## Modes of Operation
There are two main use-cases of the chat system.  One is repeater mode, which operates
on a client/server model; and thenbroadcast mode, which creates a peer-to-peer network.

### Repeater Mode
Repeater mode sets up the system to act as a server that the client can
then connect to.  Useful if you want to self-host a chat room on your own
dedicated hardware.


### Broadcast Mode
Broadcast mode is used for chatting with other clients on the local network, i,e;
the client also acts as a server for everybody else on the sub-net while actively
searching for other servers on the net.


# The Code

## Import Modules

  - ```socket```: The main technology the system is built upon.
  - ```threading```: Used for resource management involved with running the socket.
  - ```argparse.ArgumentParser```: For providing command-line functionality.
  - ```base64.b64encode```: Produces serialized byte-objects from arbitrary input.
  - ```base64.b64decode```: Re-Constitutes serialized byte-objects in machine readable format.
  - ```json.loads```: Used to convert re-constituted objects into standard data formats.
  - ```logging.getLogger```: Retrieves local logging configurations.
  - ```logging.exception```: Error handling for logging statements.
  - ```logging.info```: Logs general information a user should know.
  - ```logging.debug```: Logs everything a developer should know.

In [ ]:
import socket
import threading
from os import getlogin as username
from datetime import datetime
import inspect
from argparse import ArgumentParser
from json import loads as load
from base64 import b64decode, b64encode
from logging import getLogger, exception
from logging import info as information
from logging import debug as debugging

## Server
The server class handles the routing of packets among connected clients.

One key difference between the standard version and the lab implementation is the use of base64 to transfer communications as a serialized packet of data instead of a raw character string.

This allows us to re-build communique with extra meta-data to provide functionality.

In [ ]:
class server:

    def __init__(self, address="127.0.0.1", port=5190):
        """
        Set up a couple of constants required at runtime.  Here we've got self; but also
        we've got self.address which defines an IP to make the service available on, a self.port
        which narrows down the channel to operate on, and self.connections which enumerates every
        address that has made a valid connection to the instance.
        """

        # The classes semblance of identity.
        self             = self

        # An address to host services on.
        self.address     = address

        # A socket to listen out for.
        self.port        = port

        # A list of clients connected to this server.
        self.connections = []


    def handle_user_connection(self, connection: socket.socket, address: str) -> None:
        while True:
            try:
                # Recieve client's posted message, up to 4096 bytes.
                msg = connection.recv(4096)
    
                if msg:
                    # TODO: Log client messages to server database.

                    # Convert the msg bytes into a dictionary named message.
                    message = load(b64decode(msg)\
                                         .decode()\
                                         .replace("'", '"'))
                    
                    # Pass the dictionary to the broadcast function.
                    self.broadcast(message, connection)
    
                # Close connection if no message was sent
                else:
                    self.remove_connection(connection)
                    break
    
            except Exception as error:
                exception(
                        f"There was an issue handling the user connection:\n"
                        f"{error}"
                )
                self.remove_connection(connection)
                break
    
    
    def broadcast(self, message: dict, sender: socket.socket) -> None:
    
        # Iterate on connections in order to send message to all client's connected
        for client in self.connections:

            # Don't send back to the sender.
            if client != sender:
    
                if message["subject"]:# Add the subject line, if one is provided.
                    message_string = ( f"from: {message['username']}\n"
                                       f"subject: {message['subject']}\n"
                                       f"content:\n{message['content']}\n" )\
                        .encode()
    
                # Otherwise, present the string as normal.
                else:
                    message_string = ( f"from: {message['username']}\n"
                                       f"content:\n{message['content']}\n" )\
                        .encode()
    
                try: client.send(message_string)
    
                # If that fails, you've probably got a dead socket.
                except Exception as error:
                    print('Error broadcasting message: {error}')
                    self.remove_connection(client_conn)
    
    
    def remove_connection(self, conn: socket.socket) -> None:
    
        # Check if connection exists on connections list
        if conn in self.connections:
            # Close socket connection and remove connection from connections list
            conn.close()
            self.connections.remove(conn)
    
    
    def server(self, address, port) -> None:
        """
    
        """
        
    
        try:
            socket_instance = socket.socket(
                socket.AF_INET,
                socket.SOCK_STREAM
            )

            socket_instance.bind(
                (str(address), int(port))
            )

            socket_instance.listen(4)

            # TODO: Log successful server instance.
        
            while True:

                # Accept client connection
                socket_connection, address = socket_instance.accept()

                # Add client connection to connections list
                self.connections.append(socket_connection)

                threading.Thread( target=self.handle_user_connection,
                                  args=[ socket_connection,
                                         address            ]   )\
                         .start()


        except Exception as error:
            # TODO: Exception Logging.
            pass


        finally:
            if len(connections) > 0:
                for conn in connections:
                    remove_connection(conn)

            socket_instance.close()

## Client

In [ ]:
import socket, threading
from glob import glob
import base64
import argparse

from logging import getLogger, exception
from logging import debug as debugging
from logging import info as information
from datetime import datetime
import inspect


def createUserDB():
    """
    Create an sqlite3 database within the users .local directory and populate
    it with a table for logging chat messages.

    The table will consist of a X fields; username for logging who sent the
    message, subject for the general idea behind the message which can be set
    with a /command, a timestamp for when it was sent, and the text body
    of the message itself.
    """

    # Set up logging.
    getLogger()

    _name = inspect.stack()[0][3]
    _time = lambda datetime.timestamp(datetime.now())

    debugging(#####} CONSTANTS
        f"{_time}:{_name}:Beginning constants setup."
    )

    # The users .local directory; see the Linux FS for info about that.
    local_directory = f"/home/{username()}/.local"
    debugging(f"{_time}:{_name}:    --Local Directory: ✅")

    # Filename and path for the sqlite database.
    lab_database = f"{local_directory}/lab-93.db"
    debugging(f"{_time}:{_name}:    --Lab Database: ✅")

    # Bash command for creating the .local directory.
    createSubDirectory = f"mkdir -p {local_directory} > /dev/null "
    debugging(f"{_time}:{_name}:    --Sub-Directory Shell String: ✅")

    # SQLite3 command for creating the messages table
    # with our required columns.
    createMessagesTable_SQL = (
        f"CREATE TABLE IF NOT EXISTS "
            f"messages("
                f"username TEXT REQUIRED KEY, "
                f"subject TEXT, "
                f"timestamp REAL, "
                f"message TEXT"
            f")"
    )
    debugging(f"{_time}:{_name}:    --Messages Table Creation SQL String: ✅")



    # Check for the .local directory and create it if need be.
    debugging(f"{_time}:{_name}:    --Validate Local Directory:")

    # If glob returns any results then the directory already exists.
    if len(glob(local_directory)) >= 1:
        debugging(f"{_time}:{_name}:      --Directory exists. ✅"); pass

    else:# If not, then the directory needs to be created.
        debugging(f"{_time}:{_name}:      --Directory does not exist; creating.")

        try: subprocess.Run(# Execute the directory creation one-liner.
            createSubDirectory.split()
        ); information(
            f"{_time:{_name}:      --Created subdirectory at {local_directory}"
        )

        # Log any mishaps and inform the caller.
        except Exception as error:
            exception(
                f"{_time}:{_name}:"
                f"There was an issue creating the .local subdirectory:\n"
                f"{error}"
            )


    # Begin sqlite database connection and initialize cursor.
    debugging(f"{_time}:{_name}:    --Attempting connection to database at {lab_database}")
    try:
        # Establish connection to .db file.
        connection = sqlite3.connect(lab_database); debugging(
            f"{_time}:{_name}:      --Connection established ✅"
        )

        # Create cursor and lable it for execution.
        cursor = connection.cursor(); execute = cursor.execute; debugging(
            f"{_time}:{_name}:      --Cursor successfully labelled ✅"
        )

    # Run the table creation sql and save your work!
    while True: try:
        cursor.execute(
            createMessagesTable_SQL
        ); information(
            f"{_time}:{_name}:"
            f"Messages table successfully created for user {username()}"
        ); connection.commit(); break

    # Handle any errors gracefully, and inform the caller.
    except Exception as error:
        exception(
            f"{_time}:{_name}:"
            f"There was an issue creating a messages table "
            f"within the user database;\n{error}}"
        )

        return error


def handle_messages(connection: socket.socket):

    while True:
        try:
            msg = connection.recv(1024)

            if msg:
                print(msg.decode())
            else:
                connection.close()
                break

        except Exception as e:
            print(f'Error handling message from server: {e}')
            connection.close()
            break

def client(address, port) -> None:


    try:

        # Instantiate socket and start connection with server
        socket_instance = socket.socket()
        socket_instance.connect((SERVER_ADDRESS, int(SERVER_PORT)))

        # Create a thread in order to handle messages sent by server
        threading.Thread(target=handle_messages, args=[socket_instance]).start()

        while True:

            # Recieve user input as message string.
            message_packet = {"content": input(), "username": username()})
           

            # User Commands
            """ User commands allow for actions to be made from chat;
            such as quitting the session or defining a subject for
            a message.  Commands are defined by typing a forward slash
            as the first letter of your message."""

            # All commands start with a '/', so check for that.
            if message_packet["content"][0] != "/": pass
            else:

                # The command is the first word in the msg.
                cmd = message_packet["content"].split(" ")[0]

                # The quit command breaks the loop and
                # returns control back to the terminal.
                if cmd == "/quit": break

                # Allows the attachment of a subject line to a msg.
                if cmd == "/subject":
                    message_packet["subject"] = str(msg.split(" ")[1])
                    message_packet["content"] = str(" ".join(msg.split(" ")[2:-1]))


            # Convert message packet to ascii string
            socket_instance.send(
                base64.b64encode(
                    str(message_packet).encode('ascii')
                )
            )

        # Close connection with the server
        socket_instance.close()

    except Exception as e:
        print(f'Error connecting to server socket {e}')
        socket_instance.close()




## Argument Handling

In [ ]:
if __name__ == "__main__":


    class OptionsError(Exception):


        class ConflictingOptions(Exception, *options):
            """ Called if the user calls exclusive, conflicting options. """
            
            print(
                f"Conflicting arguments {argument for argument in options} "
                f"can not be used together."
            )

            exit()

        class InsufficientOptions(Exception,  *options, **solutions):
            """ Called if the user does not provide enough information. """

            for option in options:
                print(
                    f"Option {option} requires additional information;\n"
                    f"Try {solutions[option]}"
                )

            exit()


    parser = ArgumentParser()
    
    parser.add_argument( "-a",
                         "--address",
                         help="IP Address to route connections on." )

    parser.add_argument( "-b",
                         "--broadcast",
                         action="store_true",
                         help="Set the client to act as a dynamic agent." )

    parser.add_argument( "-c",
                         "--client",
                         action="store_true",
                         help="If using repeater mode, sets the agent to client mode." )

    parser.add_argument( "-m",
                         "--mode",
                         help="Alternate definition of either repeater or broadcast mode." )

    parser.add_argument( "-p",
                         "--port",
                         help="Port number to listen on." )

    parser.add_argument( "-r",
                         "--repeater",
                         action="store_true",
                         help="Set client to run in client/server mode." )

    parser.add_argument( "-s",
                         "--server",
                         action="store_true",
                         help="If using repeater mode, sets the agent to server mode." )

    arguments = parser.parse_args()


    # Collect or define address; defaults to all public addresses.
    if arguments.address: address = arguments.address
    else: address = "0.0.0.0"


    # Collect or define port number; default to 5190 because AOL Chat.
    if arguments.port: port = arguments.port
    else: port = 5190
    

    # The mode could be anything, but we only respond to two specific options.
    if arguments.mode and arguments.mode != "repeater" or "broadcast":
        raise OptionsError.ImproperOptions()

    # If not by mode bu by flag, but we have both flags, thats a conflict.
    elif not arguments.mode and arguments.broadcast and arguments.repeater:
        raise OptionsError.ConflictingOptions(
            "--broadcast", "--repeater"
        )

    # If not by mode nor flag, that's an insufficiency.
    elif not arguments.mode and not arguments.broadcast and not arguments.repeater:
        raise OptionsError.InsufficientOptions()


    # Broadcast-Mode subchecks.
    if arguments.broadcast is True or arguments.mode is "broadcast": ...


    # Repeater-Mode subchecks.    
    if arguments.repeater is True or arguments.mode is "repeater":

        # Check client & server aren't both given.
        if arguments.server and arguments.client:
            raise OptionsError.ConflictingOptions(
                "--client", "--server"
            )
        
        # Check at least server or client are given.
        elif not arguments.server and not arguments.client:
            raise OptionsError.InsufficientOptions(
                "repeater",
                repeater="supplying either the --server or --client flags."
            )

        else:
            if arguments.server: server(address, port)
            if arguments.client: client(address, port)